In [ ]:
!pip install transformers
!pip install llama-index-embeddings-huggingface-optimum

  Using cached transformers-4.42.3-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached safetensors-0.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_1

In [ ]:
!pip install optimum
!pip install onnx
!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/



Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/


In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
from pathlib import Path
from transformers import Pipeline
import torch.nn.functional as F
import torch

model_dir = "/huggingface/sentence_transformers"
model_name = "bge-small-zh-v1.5"

model_path = os.path.join(model_dir, model_name)
model_path

'/huggingface/sentence_transformers/bge-small-zh-v1.5'

## Generate ONNX Model

In [ ]:
onnx_path = "./tmp/bge_onnx"
# load vanilla transformers and convert to onnx
model = ORTModelForFeatureExtraction.from_pretrained(model_path, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: BertModel *****
Using framework PyTorch: 2.3.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


/tmp/tmprey_qwo1/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcbf415370>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcbf415770>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcc0a0baf0>]
/tmp/tmprey_qwo1/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb9672ab0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb9672b30>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb9672bb0>]


('./tmp/bge_onnx/tokenizer_config.json',
 './tmp/bge_onnx/special_tokens_map.json',
 './tmp/bge_onnx/vocab.txt',
 './tmp/bge_onnx/added_tokens.json',
 './tmp/bge_onnx/tokenizer.json')

In [ ]:
# copied from the model card
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


class SentenceEmbeddingPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        # we don't have any hyperameters to sanitize
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs):
        encoded_inputs = self.tokenizer(
            inputs, padding=True, truncation=True, return_tensors="pt"
        )
        return encoded_inputs

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        return {"outputs": outputs, "attention_mask": model_inputs["attention_mask"]}

    def postprocess(self, model_outputs):
        # Perform pooling
        sentence_embeddings = mean_pooling(
            model_outputs["outputs"], model_outputs["attention_mask"]
        )
        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings


vanilla_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer, device=0)

# run inference
pred = vanilla_emb("Could you assist me in finding my lost card?")

# print an excerpt from the sentence embedding
print(pred[0][:5])

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


/tmp/tmprey_qwo1/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb9672ab0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb96737b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcc015e5f0>]
tensor([ 0.0472,  0.0101,  0.0006, -0.0082, -0.0220])


2024-07-06 11:32:52.479354501 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-07-06 11:32:52.480349243 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-07-06 11:32:52.480358848 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


## Optimize Model

In [ ]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

model = ORTModelForFeatureExtraction.from_pretrained(onnx_path, file_name="model.onnx")

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model)
optimization_config = OptimizationConfig(
    optimization_level=99
)  # enable all optimizations

# apply the optimization configuration to the model
optimizer.optimize(
    save_dir=onnx_path,
    optimization_config=optimization_config,
)

/mnt/llm/feiyue/miniconda3/envs/onnx/lib/python3.11/site-packages/optimum/onnxruntime/configuration.py:779: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...


tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d9a0b70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d9a2a70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d9a38f0>]
tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc579d6870>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc579d5eb0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc579d4ab0>]


2024-07-06 11:43:44.402180201 [W:onnxruntime:, inference_session.cc:1978 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.
Configuration saved in tmp/bge_onnx/ort_config.json
Optimized model saved at: tmp/bge_onnx (external data format: False; saved all tensor to one file: True)


PosixPath('tmp/bge_onnx')

In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction

# load optimized model
model = ORTModelForFeatureExtraction.from_pretrained(
    onnx_path, file_name="model_optimized.onnx"
)

# create optimized pipeline
optimized_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer, device=0)
pred = optimized_emb("Could you assist me in finding my lost card?")
print(pred[0][:5])

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


tmp/bge_onnx/model_optimized.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcbcebcff0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcbcebcab0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcb9687f70>]
tmp/bge_onnx/model_optimized.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc57c34370>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc57c34230>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc57c34530>]
tensor([ 0.0472,  0.0101,  0.0006, -0.0082, -0.0220])


2024-07-06 11:43:54.030874495 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-07-06 11:43:54.031722394 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-07-06 11:43:54.031731398 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


## Apply ORTQuantizer

In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

model = ORTModelForFeatureExtraction.from_pretrained(
    onnx_path, file_name="model_optimized.onnx"
)

# create ORTQuantizer and define quantization configuration
dynamic_quantizer = ORTQuantizer.from_pretrained(model)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
model_quantized_path = dynamic_quantizer.quantize(
    save_dir=onnx_path,
    quantization_config=dqconfig,
)

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)


tmp/bge_onnx/model_optimized.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc758801b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdcbce62df0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4f4c9330>]


Quantizing model...


RuntimeError: Unable to find data type for weight_name='/encoder/layer.0/attention/self/query/MatMul_output_0'. shape_inference failed to return a type probably this node is from a different domain or using an input produced by such an operator. This may happen if you quantize a model already quantized. You may use extra_options `DefaultTensorType` to indicate the default weight type, usually `onnx.TensorProto.FLOAT`.

In [ ]:
import torch
import numpy as np
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Set, Tuple, Union
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers.modeling_outputs import BaseModelOutput


class MyORTModelForFeatureExtraction(ORTModelForFeatureExtraction):
    def forward(
        self,
        input_ids: Optional[Union[torch.Tensor, np.ndarray]] = None,
        attention_mask: Optional[Union[torch.Tensor, np.ndarray]] = None,
        token_type_ids: Optional[Union[torch.Tensor, np.ndarray]] = None,
        **kwargs,
    ):
        use_torch = isinstance(input_ids, torch.Tensor)
        self.raise_on_numpy_input_io_binding(use_torch)

        if self.device.type == "cuda" and self.use_io_binding:
            io_binding, output_shapes, output_buffers = self.prepare_io_binding(
                input_ids,
                attention_mask,
                token_type_ids,
                ordered_input_names=self._ordered_input_names,
            )

            # run inference with binding & synchronize in case of multiple CUDA streams
            io_binding.synchronize_inputs()
            self.model.run_with_iobinding(io_binding)
            io_binding.synchronize_outputs()

            last_hidden_state = output_buffers["last_hidden_state"].view(
                output_shapes["last_hidden_state"]
            )
        else:
            model_inputs = {
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "token_type_ids": token_type_ids,
            }

            onnx_inputs = self._prepare_onnx_inputs(use_torch, **model_inputs)
            onnx_outputs = self.model.run(None, onnx_inputs)
            model_outputs = self._prepare_onnx_outputs(use_torch, *onnx_outputs)

            if "last_hidden_state" in self.output_names:
                last_hidden_state = model_outputs["last_hidden_state"]
            else:
                # TODO: This allows to support sentence-transformers models (sentence embedding), but is not validated.
                last_hidden_state = next(iter(model_outputs.values()))

        # converts output to namedtuple for pipelines post-processing
        return BaseModelOutput(last_hidden_state=last_hidden_state)

In [ ]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

model = MyORTModelForFeatureExtraction.from_pretrained(
    onnx_path, file_name="model_quantized.onnx", provider="CUDAExecutionProvider"
)

embed_model = OptimumEmbedding(folder_name="./tmp/bge_onnx", model=model)

tmp/bge_onnx/model_quantized.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d9481b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d966230>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fdc4d967170>]


2024-07-06 11:53:29.724101597 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 100 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-07-06 11:53:29.726186251 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-07-06 11:53:29.726195987 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:
%%time
for i in range(5000):
    embeddings = embed_model.get_text_embedding("It is raining cats and dogs here!")
# embeddings

KeyboardInterrupt: 